# Physical Modelling

Port of [Think Complexity chapter ](http://greenteapress.com/complexity2/html/index.html) by Allen Downey.

In [ ]:
using Luxor

## Diffusion

In 1952 Alan Turing published a paper called “The chemical basis of morphogenesis”, which describes the behavior of systems involving two chemicals that diffuse in space and react with each other. He showed that these systems produce a wide range of patterns, depending on the diffusion and reaction rates, and conjectured that systems like this might be an important mechanism in biological growth processes, particularly the development of animal coloration patterns.

Turing’s model is based on differential equations, but it can be implemented using a cellular automaton.

Before we get to Turing’s model, we’ll start with something simpler: a diffusion system with just one chemical. We’ll use a 2-D CA where the state of each cell is a continuous quantity (usually between 0 and 1) that represents the concentration of the chemical.

We’ll model the diffusion process by comparing each cell with the average of its neighbors. If the concentration of the center cell exceeds the neighborhood average, the chemical flows from the center to the neighbors. If the concentration of the center cell is lower, the chemical flows the other way.

We’ll use a diffusion constant, `r`, that relates the difference in concentration to the rate of flow:

In [ ]:
function applydiffusion(array::Array{Float64, 2}, r::Float64=0.1)
    nr_y, nr_x = size(array)
    out = deepcopy(array)
    for y in 2:nr_y-1
        for x in 2:nr_x-1
            c = array[y-1, x] + array[y, x-1] + array[y, x+1] + array[y+1, x] - 4*array[y, x]
            out[y, x] += r*c
        end
    end
    out
end

visualisation:

In [ ]:
function visualizearray(array::Array{Float64, 2}, dim)
    (nr_y, nr_x) = size(array)
    width = dim * (nr_x - 1)
    height = dim * (nr_y - 1)
    Drawing(width, height, "out.svg")
    for (j, y) in enumerate(2:nr_y-1)
        for (i, x) in enumerate(2:nr_x-1)
            sethue(setgray(0.8*(1-array[y, x])+0.1))
            box(i*dim, j*dim, dim, dim, :fill)
        end
     end
     finish()
     preview()
end

In [ ]:
array = zeros(Float64, 11, 11)
array[5:7, 5:7] = ones(Float64, 3, 3)
visualizearray(array, 16)

In [ ]:
for i in 1: 10
    array = applydiffusion(array)
    visualizearray(array, 16)
    sleep(1)
end


## Reaction-Diffusion

Now let’s add a second chemical.

In [ ]:
function applyreactiondiffusion(
        a::Array{Float64, 2}, 
        b::Array{Float64, 2}, 
        ra::Float64=0.5, rb=Float64=0.25, 
        f::Float64=0.055, k = Float64=0.062)
    nr_y, nr_x = size(a)
    a_out = deepcopy(a)
    b_out = deepcopy(b)
    for y in 2:nr_y-1
        for x in 2:nr_x-1
            reaction = a[y, x] * b[y, x]^2
            ca = 0.25*(a[y-1, x] + a[y, x-1] + a[y, x+1] + a[y+1, x]) - a[y, x]
            cb = 0.25*(b[y-1, x] + b[y, x-1] + b[y, x+1] + b[y+1, x]) - b[y, x]
            a_out[y, x] += ra*ca - reaction + f * (1 - a[y, x])
            b_out[y, x] += rb*cb + reaction - (f+k) * b[y, x]
        end
    end
    a_out, b_out
end

- `ra`: The diffusion rate of A (analogous to r in the previous section).
- `rb`: The diffusion rate of B. In most versions of this model, rb is about half of ra.
- `f`: The “feed” rate, which controls how quickly A is added to the system.
- `k`: The “kill” rate, which controls how quickly B is removed from the system.

`ca` and `ca` are the result of applying a diffusion kernel to A and B. Multiplying by `ra` and `rb` yields the rate of diffusion into or out of each cell.

The term `a*b^2`represents the rate that A and B react with each other. Assuming that the reaction consumes A and produces B, we subtract this term in the first equation and add it in the second.

The term `f * (1-a)` determines the rate that A is added to the system. Where A is near 0, the maximum feed rate is `f`. Where A approaches 1, the feed rate drops off to zero.

Finally, the term `(f+k) * b` determines the rate that B is removed from the system. As B approaches 0, this rate goes to zero.

As long as the rate parameters are not too high, the values of A and B usually stay between 0 and 1.

In [ ]:
a = ones(Float64, 258, 258)
b = rand(Float64, 258, 258)*0.1
b[129-12:129+12, 129-12:129+12] += ones(Float64, 25, 25)*0.1
visualizearray(b, 2)

In [ ]:
# f = 0.035, 0.055, 0.039 k = 0.057, 0.062, 0.065
for i in 1:4000
    a, b = applyreactiondiffusion(a, b, 0.5, 0.25, 0.035, 0.057)
    if mod(i, 500) == 0
        visualizearray(b, 2)
    end
end

Since 1952, observations and experiments have provided some support for Turing’s conjecture. At this point it seems likely, but not yet proven, that many animal patterns are actually formed by reaction-diffusion processes of some kind.

## Percolation

Percolation is a process in which a fluid flows through a semi-porous material. Examples include oil in rock formations, water in paper, and hydrogen gas in micropores. Percolation models are also used to study systems that are not literally percolation, including epidemics and networks of electrical resistors.

We’ll explore a 2-D CA that simulates percolation.

- Initially, each cell is either “porous” with probability `q` or “non-porous” with probability `1-q`.
- When the simulation begins, all cells are considered “dry” except the top row, which is “wet”.
- During each time step, if a porous cell has at least one wet neighbor, it becomes wet. Non-porous cells stay dry.
- The simulation runs until it reaches a “fixed point” where no more cells change state.

If there is a path of wet cells from the top to the bottom row, we say that the CA has a “percolating cluster”.

Two questions of interest regarding percolation are (1) the probability that a random array contains a percolating cluster, and (2) how that probability depends on `q`.

In [ ]:
function applypercolation(array::Array{Float64, 2})
    nr_y, nr_x = size(array)
    out = deepcopy(array)
    for y in 2:nr_y-1
        for x in 2:nr_x-1
            if out[y, x] > 0.0
                c = array[y-1, x] + array[y, x-1] + array[y, x+1] + array[y+1, x]
                if c ≥ 0.5
                    out[y, x] = 0.5
                end
            end
        end
    end
    out
end

visualisation:

In [ ]:
function createwall(n, q)
    array = zeros(Float64, n+2, n+2)
    array[2, 2:n+1] = ones(Float64, n)*0.5
    array[3:n+1, 2:n+1] = rand(Float64, n-1, n)
    for y in 3:n+1
        for x in 2:n+1
            if array[y, x] < q
                array[y, x] = 0.1
            else
                array[y, x] = 0.0
            end
        end
    end
    array
end
array = createwall(100, 0.5)
visualizearray(array, 4)

In [ ]:
for i in 1:100
    array = applypercolation(array)
end
visualizearray(array, 4)

## Phase Change

Now let’s test whether a random array contains a percolating cluster.

In [ ]:
function testpercolation(array::Array{Float64, 2}, vis=false)
    numberwet = count(x->x==0.5, array[3:101, 2:101])
    while true
        array = applypercolation(array)
        if count(x->x==0.5, array[101, 2:101]) > 0
            if vis; visualizearray(array, 4) end
            return true
        end
        newnumberwet = count(x->x==0.5, array[3:101, 2:101])
        if numberwet == newnumberwet
            if vis; visualizearray(array, 4) end
            return false
        end
        numberwet = newnumberwet
    end
end

In [ ]:
array = createwall(100, 0.5)
testpercolation(array, true)

To estimate the probability of a percolating cluster, we generate many random arrays and test them.

In [ ]:
function estimateprob(;n=100, q=0.5, iters=100)
    t = Bool[]
    for _ in 1:iters
        array = createwall(n, q)
        push!(t, testpercolation(array))
    end
    count(x->x, t) / iters
end

In [ ]:
estimateprob(q = 0.5)

We can estimate the critical value more precisely using a random walk. Starting from an initial value of `q`, we construct a wall and check whether it has a percolating cluster. If so, `q` is probably too high, so we decrease it. If not, `q` is probably too low, so we increase it.

In [ ]:
function findcritical(;n=100, q=0.5, iters=100)
    qs = [q]
    for _ in 1:iters
        array = createwall(n, q)
        if testpercolation(array)
            q -= 0.004
        else
            q += 0.004
        end
        push!(qs, q)
    end
    qs
end

In [ ]:
findcritical()

With `n=100` the mean of `qs` is about 0.59; this value does not seem to depend on `n`.

The rapid change in behavior near the critical value is called a phase change by analogy with phase changes in physical systems, like the way water changes from liquid to solid at its freezing point.

A wide variety of systems display a common set of behaviors and characteristics when they are at or near a critical point. These behaviors are known collectively as critical phenomena.

## Fractals

To understand fractals, we have to start with dimensions.

For simple geometric objects, dimension is defined in terms of scaling behavior. For example, if the side of a square has length `l`, its area is `l^2`. The exponent, 2, indicates that a square is two-dimensional. Similarly, if the side of a cube has length `l`, its volume is `l^3`, which indicates that a cube is three-dimensional.

More generally, we can estimate the dimension of an object by measuring some kind of size (like area or volume) as a function of some kind of linear measure (like the length of a side).

As an example, I’ll estimate the dimension of a 1-D cellular automaton by measuring its area (total number of “on” cells) as a function of the number of rows.

In [ ]:
function inttorule1dim(val::UInt8)
    digs = BitArray(digits(val, base=2))
    for i in length(digs):7
        push!(digs, false)
    end
    digs
end

function applyrule1dim(rule::BitArray{1}, bits::BitArray{1})
    val = 1 + bits[3] + 2*bits[2] + 4*bits[1]
    rule[val]
end

function step1dim(x₀::BitArray{1}, rule::BitArray{1}, steps::Int64)
    xs = [x₀]
    len = length(x₀)
    for i in 1:steps
        x = copy(x₀)
        for j in 2:len-1
            x[j] = applyrule1dim(rule, xs[end][j-1:j+1])
        end
        push!(xs, x)
    end
    xs
end

function visualize1dim(res, dim)
    width = dim * (length(res[1]) + 1)
    height = dim * (length(res) + 1)
    Drawing(width, height, "out.svg")
    for (i, arr) in enumerate(res)
        for (j, val) in enumerate(arr)
            if val
                sethue("grey")
            else
                sethue("lightgrey")
            end
            box(j*dim, i*dim, dim, dim, :fill)
        end
     end
     finish()
     preview()
end

In [ ]:
x₀ = falses(65)
x₀[33] = true
res = step1dim(x₀, inttorule1dim(UInt8(18)), 31) # 20, 50, 18
visualize1dim(res, 5)

I’ll estimate the dimension of these CAs with the following function, which counts the number of on cells after each time step.

In [ ]:
function countcells(rule, n=501)
    x₀ = falses(2*n+3)
    x₀[n+2] = true
    res = step1dim(x₀, inttorule1dim(UInt8(rule)), n)
    cells = [1]
    for i in 2:n
        push!(cells, cells[end]+sum(line->count(cell->cell, line), res[i]))
    end
    cells
end

In [ ]:
using Pkg
pkg"add Plots"

using Plots

In [ ]:
n = 501
rule = 18 # 20, 50, 18
res = countcells(rule, n);

In [ ]:
plot(1:n, 1:n, xaxis=:log, yaxis=:log, label="d=1")
plot!(1:n, (1:n).^2, xaxis=:log, yaxis=:log, label="d=2")
plot!(1:n,res, xaxis=:log, yaxis=:log, label="rule $rule")

Rule 20 (left) produces 3 cells every 2 time steps, so the total number of cells after $i$ steps is $y = 1.5i$. Taking the log of both sides, we have $\log y = \log 1.5 + \log i$, so on a log-log scale, we expect a line with slope 1. In fact, the estimated slope of the line is 1.01.

Rule 50 (center) produces i+1 new cells during the ith time step, so the total number of cells after $i$ steps is $y = i^2 + i$. If we ignore the second term and take the log of both sides, we have $\log y \approx 2 \log i$, so as $i$ gets large, we expect to see a line with slope 2. In fact, the estimated slope is 1.97.

Finally, for Rule 18 (right), the estimated slope is about 1.57, which is clearly not 1, 2, or any other integer. This suggests that the pattern generated by Rule 18 has a “fractional dimension”; that is, it is a fractal.

This way of estimating a fractal dimension is called box-counting.

## Fractals and Percolation

Now let’s get back to percolation models.

To estimate their fractal dimension, we can run CAs with a range of sizes, count the number of wet cells in each percolating cluster, and then see how the cell counts scale as we increase the size of the array.

In [ ]:
function percolationwet(array::Array{Float64, 2})
    numberwet = count(x->x==0.5, array[3:end-1, 2:end-1])
    while true
        array = applypercolation(array)
        if count(x->x==0.5, array[end-1, 2:end-1]) > 0
            break
        end
        newnumberwet = count(x->x==0.5, array[3:end-1, 2:end-1])
        if numberwet == newnumberwet
            break
        end
        numberwet = newnumberwet
    end
    numberwet
end

In [ ]:
res = Float64[]
sizes = 10:10:200
q = 0.60
for size in sizes
    array = createwall(size, q)
    push!(res, percolationwet(array))
end
plot(sizes, sizes, xaxis=:log, yaxis=:log, label="d=1")
plot!(sizes, (sizes).^2, xaxis=:log, yaxis=:log, label="d=2")
plot!(sizes, res, xaxis=:log, yaxis=:log, seriestype=:scatter, label="q = $q")

The dots show the number of cells in each percolating cluster. The slope of a line fitted to these dots is often near 1.85, which suggests that the percolating cluster is, in fact, fractal when `q` is near the critical value.

When `q` is larger than the critical value, nearly every porous cell gets filled, so the number of wet cells is close to `q * size^2`, which has dimension 2.

When `q` is substantially smaller than the critical value, the number of wet cells is proportional to the linear size of the array, so it has dimension 1.